In [5]:
import numpy as np
import pandas as pd
import os
import json
import re
from tqdm import tqdm

news_data = pd.read_csv('naver_news_all.csv')
news_data.head(5)

,title,article,link,subject
0,"연금에 꽂힌 김동원… 한화생명, 디지털 판매회사 세운다",한화생명이 디지털 연금 판매전문회사를 설립하기 위한 초읽기에 들어갔다. ...,https://news.naver.com/main/read.naver?mode=LS...,"""개인연금"" -국민연금 -퇴직연금 -중국 -영국 -콴텍 -출산 -게임 -인사"
1,"[단독] ""연금에서 답 찾자"" 한화생명, 디지털 판매회사 세운다",한화생명이 디지털 연금 전문회사를 설립하기 위해 인원 충원 작업에 들어갔...,https://news.naver.com/main/read.naver?mode=LS...,"""개인연금"" -국민연금 -퇴직연금 -중국 -영국 -콴텍 -출산 -게임 -인사"
2,연금저축 작년 신규계약 194% 폭증…2030세대 지갑 열었다,금융감독원. 연합뉴스 지난해 20~30대의 연금저축 신규 ...,https://news.naver.com/main/read.naver?mode=LS...,"""개인연금"" -국민연금 -퇴직연금 -중국 -영국 -콴텍 -출산 -게임 -인사"
3,CD금리 대안 KOFR 뜬다,3개월 선물 이어 ETF 상장거래 많은 RP시장 기반 산정CD금리보다 시장상황 ...,https://news.naver.com/main/read.naver?mode=LS...,"""개인연금"" -국민연금 -퇴직연금 -중국 -영국 -콴텍 -출산 -게임 -인사"
4,"[머니팁]미래에셋증권, 개인연금 이전하면 10만원대 경품",50세 이상 개인연금 이전 시 최대 300만원 상품권[이데일리 이지현 기자] 미...,https://news.naver.com/main/read.naver?mode=LS...,"""개인연금"" -국민연금 -퇴직연금 -중국 -영국 -콴텍 -출산 -게임 -인사"


In [7]:
news_data.isnull().sum()
news_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1513 entries, 0 to 1512
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    1513 non-null   object
 1   article  1513 non-null   object
 2   link     1513 non-null   object
 3   subject  1513 non-null   object
dtypes: object(4)
memory usage: 47.4+ KB


In [ ]:
'"개인연금" -국민연금 -퇴직연금 -중국 -영국 -콴텍 -출산 -게임 -인사',
            '"개인연금" +ETF -TDF -퇴직연금', 
            '"연금상품" +고르는 +증권',
            'IRP',
            '변액연금보험',
            '"개인연금" +연령 -중국 -국민연금 -퇴직연금',
            '"개인연금" +수익률 -국민연금 -퇴직연금',
            '개인연금 +사망 -복권 -출산 -국민연금 -퇴직연금 -중국 -농지연금 -주택연금',
            '"연금계좌" +이전 -비트코인 -국민연금 -스웨덴',
            '개인연금 +세대 -중국 -국민연금 -치킨 -농지연금 -주택연금',
            '"irp" +증권사',
            '"개인연금" +노후대비 -퇴직연금 -주택연금 -중국 -농지연금 -기초연금',
            '퇴직연금 +수령방법 -국민연금', '연금저축 +수령방법 -국민연금 +절세', '연금저축 +절세 -국민연금',
            '퇴직연금 +중도인출', '연금저축 +중도인출', '"퇴직연금" +중도해지', '"개인연금" +중도해지 -주택연금'
            '개인연금 연금저축 "원금보장" -국민연금', '퇴직연금 "원금보장" -국민연금',
            'irp 연금저축 +차이 -국민연금',
            '"연금저축" +납입한도 -국민연금', '"퇴직연금" +납입한도 -국민연금"',
            '"연금" 세제혜택 세액공제 -국민연금',

'"퇴직연금" -국민연금 -피델리티 -중국 -영국 -콴텍 -출산 -게임 -인사',
        '"퇴직연금" +ETF -TDF -개인연금 -콴텍',
        '퇴직연금 "해지" -개인연금',
        '퇴직연금 수익률 -국민연금 -개인연금','퇴직연금 +사망 -개인연금',
        '퇴직연금 +세대 -피델리티 -중국 -국민연금 -치킨 -농지연금 -주택연금',
        '"퇴직연금" +노후대비 -주택연금 -중국 -농지연금',
        '개인연금 +운용방법 -퇴직연금 -국민연금 -사학연금', 'irp +운용방법 -국민연금 -사학연금',
        '연금저축 +운용방법 -퇴직연금 -국민연금', '개인연금 +펀드종목 -퇴직연금 -국민연금',
        '퇴직연금 +펀드종목 -개인연금 -국민연금', '연금저축 +펀드종목 -퇴직연금',
        '"연금" +증권 +가입서류 -국민연금'

In [16]:
news_data.replace({'"개인연금" -국민연금 -퇴직연금 -중국 -영국 -콴텍 -출산 -게임 -인사': '개인연금',
                                '"개인연금" +ETF -TDF -퇴직연금': '개인연금 ETF',
                                '"연금상품" +고르는 +증권': '연금상품 고르는 방법',
                                '"개인연금" +연령 -중국 -국민연금 -퇴직연금': '개인연금 연령',
                                '"개인연금" +수익률 -국민연금 -퇴직연금': '개인연금 수익률',
                                '개인연금 +사망 -복권 -출산 -국민연금 -퇴직연금 -중국 -농지연금 -주택연금': '개인연금 사망',
                                '"연금계좌" +이전 -비트코인 -국민연금 -스웨덴': '연금계좌 이전',
                                '개인연금 +세대 -중국 -국민연금 -치킨 -농지연금 -주택연금': 'MZ세대',
                                '"irp" +증권사': 'IRP 증권사',
                                '"개인연금" +노후대비 -퇴직연금 -주택연금 -중국 -농지연금 -기초연금': '개인연금 노후대비',
                                '퇴직연금 +수령방법 -국민연금': '퇴직연금 수령방법',
                                '연금저축 +수령방법 -국민연금 +절세': '연금저축 수령방법',
                                '연금저축 +절세 -국민연금': '연금저축 절세',
                                '퇴직연금 +중도인출': '퇴직연금 중도인출',
                                '연금저축 +중도인출': '연금저축 중도인출',
                                '"퇴직연금" +중도해지': '퇴직연금 중도해지',
                                '"개인연금" +중도해지 -주택연금': '개인연금 중도해지',
                                '개인연금 연금저축 "원금보장" -국민연금': '개인연금 원금보장',
                                '퇴직연금 "원금보장" -국민연금': '퇴직연금 원금보장',
                                'irp 연금저축 +차이 -국민연금': 'IRP 연금저축 차이',
                                '"연금저축" +납입한도 -국민연금': '연금저축 납입한도',
                                '"퇴직연금" +납입한도 -국민연금"': '퇴직연금 납입한도',
                                '"연금" 세제혜택 세액공제 -국민연금': '세제혜택',
                                '"퇴직연금" -국민연금 -피델리티 -중국 -영국 -콴텍 -출산 -게임 -인사': '퇴직연금',
                                '"퇴직연금" +ETF -TDF -개인연금 -콴텍': '퇴직연금 ETF',
                                '퇴직연금 "해지" -개인연금': '퇴직연금 해지',
                                '퇴직연금 수익률 -국민연금 -개인연금': '퇴직연금 수익률',
                                '퇴직연금 +사망 -개인연금': '퇴직연금 사망',
                                '퇴직연금 +세대 -피델리티 -중국 -국민연금 -치킨 -농지연금 -주택연금': 'MZ세대',
                                '"퇴직연금" +노후대비 -주택연금 -중국 -농지연금': '퇴직연금 노후대비',
                                '개인연금 +운용방법 -퇴직연금 -국민연금 -사학연금': '개인연금 운용방법',
                                'irp +운용방법 -국민연금 -사학연금': 'IRP 운용방법',
                                '연금저축 +운용방법 -퇴직연금 -국민연금': '연금저축 운용방법',
                                '개인연금 +펀드종목 -퇴직연금 -국민연금': '개인연금 펀드종목',
                                '퇴직연금 +펀드종목 -개인연금 -국민연금': '퇴직연금 펀드종목',
                                '연금저축 +펀드종목 -퇴직연금': '연금저축 펀드종목',
                                '"연금" +증권 +가입서류 -국민연금': '가입서류'}, inplace = True)
news_data.tail(20)

,title,article,link,subject
1493,ETF 시장가 주문 넣으면 안되는 이유 [한경제의 솔깃한 경제],"주코노미 TV - 공모주, ETF, 세금 Q&A주식투자인구 800만 시대, 아직...",https://news.naver.com/main/read.naver?mode=LS...,연금저축 펀드종목
1494,"30대 초반 신혼부부, 재테크 어떻게 해야 하나요",[GETTYIMAGES]Q 직장에서 만나 이제 막 결혼한 30대 초반 신...,https://news.naver.com/main/read.naver?mode=LS...,연금저축 펀드종목
1495,"수익률에 안정성까지… TDF, 연 10% 노후상품으로 떴다",[머니S리포트] 연금과 ‘환상궁합’… TDF에 자산배분 맡겨볼까①[편집자주]늘어...,https://news.naver.com/main/read.naver?mode=LS...,연금저축 펀드종목
1496,"30대 직장인 자산관리, 지금 제대로 하고 있나요?",[GettyImages]Q 부모로부터 독립해 혼자 오피스텔에 사는 30대...,https://news.naver.com/main/read.naver?mode=LS...,연금저축 펀드종목
1497,"하이투자증권, 안정적인 배당 수익에 시세차익까지 '일석이조'",삼성 배당주장기 펀드◆ 증권사 추천상품 ◆ 하이투자증권은 하반기 주목할...,https://news.naver.com/main/read.naver?mode=LS...,연금저축 펀드종목
1498,"삼성증권, 다이렉트IRP 1주년 기념 '천생연금 이벤트' 실시",오는 6월30일까지 진행 사진=삼성증권삼성증권은 개인형퇴직연금(IRP)에 ...,https://news.naver.com/main/read.naver?mode=LS...,가입서류
1499,월별로 보는 2021년 증권업계 이슈,코스피 3000·코스닥 1000 돌파금융당국 수장 동시 교체...고승범-정은보 ...,https://news.naver.com/main/read.naver?mode=LS...,가입서류
1500,퇴직연금 운용현황 집중평가…가입자 교육도 중시,◆ 퇴직연금대상 / 이렇게 심사했다 ◆ 지난 5월 4일 매일경제신문...,https://news.naver.com/main/read.naver?mode=LS...,가입서류
1501,"광주은행 ""'마이데이터' 가입고객 스벅커피 쿠폰 제공""",광주은행은 오는 3월 1일까지 ‘마이데이터에서 숨은 돈 찾고 커피 한잔 ...,https://news.naver.com/main/read.naver?mode=LS...,가입서류
1502,IRP 수익률 업계 최상위권…'수수료 면제' 다이렉트 서비스로 돌풍,Cover Story연금시장 판도 바꾼 삼성증권 “차별화한 포트폴리오로 최...,https://news.naver.com/main/read.naver?mode=LS...,가입서류


In [17]:
news_data.to_csv('naver_news_all2.csv', index=False, encoding='utf-8-sig')